In [ ]:
#https://www.kaggle.com/code/cliviakong/amex-lightgbm-parameter-search/edit

# Import Libraries

In [ ]:
! git clone --recursive https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 27165, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 27165 (delta 164), reused 204 (delta 127), pack-reused 26891
Receiving objects: 100% (27165/27165), 19.59 MiB | 26.22 MiB/s, done.
Resolving deltas: 100% (20092/20092), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), done.        
remote: Co

In [ ]:
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_2_2
-- Looking for CL_VERSION_2_2 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "2.2") 
-- OpenCL include directory: /usr/include
-- Found Boost: /usr/include (found suitable version "1.65.1", minimum required is "1.56.0") found components: filesystem system 
-- Performing Test MM_PREFET

In [ ]:
import gc
import lightgbm as lgb

import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

%pylab inline
import seaborn as sns

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Data

In [ ]:
%%time
features_avg = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21',
                'B_22', 'B_23', 'B_24', 'B_25', 'B_28', 'B_29', 'B_30', 'B_32', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41', 'B_42', 'D_39', 'D_41', 
                'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_50', 'D_51', 'D_53', 'D_54', 'D_55', 'D_58', 'D_59', 'D_60', 'D_61',
                'D_62', 'D_65', 'D_66', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_80', 
                'D_82', 'D_84', 'D_86', 'D_91', 'D_92', 'D_94', 'D_96', 'D_103', 'D_104', 'D_108',
                'D_112', 'D_113', 'D_114', 'D_115', 'D_117', 'D_118', 'D_119', 'D_120', 'D_121',
                'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_129', 'D_131', 
                'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_140', 'D_141', 
                'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11',
                'R_14', 'R_15', 'R_16', 'R_17', 'R_20', 'R_21', 'R_22', 'R_24', 'R_26', 'R_27',
                'S_3', 'S_5', 'S_6', 'S_7', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_18', 'S_22', 'S_23', 'S_25', 'S_26']
features_min = ['B_2', 'B_4', 'B_5', 'B_9', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_19', 'B_20', 
                'B_28', 'B_29', 'B_33', 'B_36', 'B_42', 'D_39', 'D_41', 'D_42', 'D_45', 'D_46', 'D_48', 'D_50', 'D_51',
                'D_53', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_62', 'D_70', 'D_71', 'D_74', 'D_75', 'D_78', 
                'D_83', 'D_102', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_128', 
                'D_132', 'D_140', 'D_141', 'D_144', 'D_145', 'P_2', 'P_3', 'R_1', 'R_27',
                'S_3', 'S_5', 'S_7', 'S_9', 'S_11', 'S_12', 'S_23', 'S_25']
features_max = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_21',
                'B_23', 'B_24', 'B_25', 'B_29', 'B_30', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_42', 'D_39', 'D_41',
                'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50',
                'D_52', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 
                'D_61', 'D_63', 'D_64', 'D_65', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_76', 'D_77', 'D_78', 'D_80',
                'D_82', 'D_84', 'D_91', 'D_102', 'D_105', 'D_107', 'D_110', 'D_111',
                'D_112', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_131', 
                'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_138', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145', 
                'P_2', 'P_3', 'P_4', 'R_1', 'R_3', 'R_5', 'R_6', 'R_7', 'R_8', 'R_10', 'R_11', 'R_14', 'R_17', 'R_20', 'R_26', 
                'R_27', 'S_3', 'S_5', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']
features_last = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_11',
                 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 
                 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_28', 'B_29', 'B_30',
                 'B_32', 'B_33', 'B_36', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41',
                 'B_42', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51',
                 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61',
                 'D_62', 'D_63', 'D_64', 'D_65', 'D_69', 'D_70', 'D_71',
                 'D_72', 'D_73', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81',
                 'D_82', 'D_83', 'D_86', 'D_91', 'D_96', 'D_105', 'D_106', 
                 'D_112', 'D_114', 'D_119', 'D_120', 'D_121', 'D_122', 'D_124', 'D_125', 'D_126', 'D_127', 'D_130', 'D_131', 
                 'D_132', 'D_133', 'D_134', 'D_138', 'D_140', 'D_141', 'D_142', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11',
                 'R_12', 'R_13', 'R_14', 'R_15', 'R_19', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_16', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27']

for i in ['train']:
    df = pd.read_parquet(f'/content/drive/MyDrive/Amex_Default_Prediction_Kaggle_Competition/{i}.parquet')
    cid = pd.Categorical(df.pop('customer_ID'), ordered=True)
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer
    if 'target' in df.columns:
        df.drop(columns=['target'], inplace=True)
    gc.collect()
    print('Read', i)
    df_avg = (df
              .groupby(cid)
              .mean()[features_avg]
              .rename(columns={f: f"{f}_avg" for f in features_avg})
             )
    gc.collect()
    print('Computed avg', i)
    df_min = (df
              .groupby(cid)
              .min()[features_min]
              .rename(columns={f: f"{f}_min" for f in features_min})
             )
    gc.collect()
    print('Computed min', i)
    df_max = (df
              .groupby(cid)
              .max()[features_max]
              .rename(columns={f: f"{f}_max" for f in features_max})
             )
    gc.collect()
    print('Computed max', i)
    df = (df.loc[last, features_last]
          .rename(columns={f: f"{f}_last" for f in features_last})
          .set_index(np.asarray(cid[last]))
         )
    gc.collect()
    print('Computed last', i)
    df = pd.concat([df, df_min, df_max, df_avg], axis=1)
    if i == 'train': train = df
    else: test = df
    print(f"{i} shape: {df.shape}")
    del df, df_avg, df_min, df_max, cid, last

target = pd.read_csv( '/content/drive/MyDrive/Amex_Default_Prediction_Kaggle_Competition/train_labels.csv').target.values
print(f"target shape: {target.shape}")

Read train
Computed avg train
Computed min train
Computed max train
Computed last train
train shape: (458913, 469)
target shape: (458913,)
CPU times: user 2min 25s, sys: 23.8 s, total: 2min 49s
Wall time: 2min 48s


# LightGBM baseline

In [ ]:
# LGBM key structural parameters 
max_depth = [i for i in list(range(3,20,2))]       # more levels the tree has, makes the model more complex and prone to overfit, too low, will be underfit
num_leaves = [i for i in list(range(100, 500, 50))]            # decides tree structure, controls the num of decision leaves in a tree;can estimate as 2^(max_depth), but it impacts more than max depth, be cautious
min_data_in_leaf = [i for i in list(range(10,1000,50))] 

# LGBM key parameters for accuracy , these two features also used for early stopping and set low value for learning rate
n_estimators =  [i for i in list(range(100,1800,100))]     # controls the number of decision trees
learning_rate = [i/100 for i in list(range(1,32, 2))]             # step size paratmeter of the gradient descent, ease the overfitting

# LGBM other features to control overfitting
lambda_l1 = [i for i in list(range(0, 100, 10))]    # l2 and l2 is similar with reg_lambda and reg_alpha on XGBoost, these two are not directly correlated with overfitting
lambda_l2 = [i for i in list(range(0, 100, 10))] 
bagging_fraction = [i/10 for i in list(range(1,10, 1))]    # range within (0,1) should be smaller than 1.0; trial.suggest_float('bragging_fraction", 0.2, 0.95, step = 0.1)
bagging_freq = [1]   # integer value
feature_fraction = [i/10 for i in list(range(1,10, 1))]   # range within (0,1);trial.suggest_float('bragging_fraction", 0.2, 0.95, step = 0.1)

# Other
boosting =  ['gbdt']  # gbdt is default
#https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

In [ ]:
params = {
    "max_depth": max_depth,
    "learning_rate": learning_rate,
    "num_leaves": num_leaves,
    "n_estimators": n_estimators,
    "verbose": -1,
    'objective': "binary",
    'metric': None,
   # 'metric': "logloss",
   'bagging_fraction': bagging_fraction,
   'bagging_freq': bagging_freq,
   'feature_fraction': feature_fraction,
}

#    'device_type': 'gpu',
# "n_estimators": 200,
d_train = lgb.Dataset(train.iloc[:1000], label=target[:1000], free_raw_data=False)

result = lgb.cv(params, d_train, nfold=5)
pd.DataFrame(result).tail(5)

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:648: UserWarning: Found 'n_estimators' in params. Will use it instead of 'num_boost_round' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'num_boost_round' argument")


TypeError: ignored

# GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
model = lgb.LGBMClassifier(verbose=-1, verbosity=-1)
param_dist = {
    "max_depth": max_depth,
    "min_child_weight": [1, 3, 6],
    "n_estimators": n_estimators,
    "learning_rate": np.linspace(0.01, 0.3, 3),                           #np.linspace(0.01, 0.3, 3),
    #'device_type': ['gpu']
    'bagging_fraction': bagging_fraction,
    'bagging_freq': bagging_freq,
    'feature_fraction': feature_fraction,
}

grid_search = GridSearchCV(
    model,
    param_dist,
    cv=5,
    verbose=10,
    n_jobs=-1,
    scoring="neg_log_loss",
)
grid_search.fit(train.iloc[:1000], target[:1000])

In [ ]:
grid_search.best_estimator_, grid_search.best_score_

# RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    "max_depth": max_depth,
    "min_child_weight": [1, 3, 6],
    "n_estimators": n_estimators,
    "learning_rate": np.linspace(0.01, 0.3, 3),
    #'device_type': ['gpu']
    'bagging_fraction': bagging_fraction,
    'bagging_freq': bagging_freq,
    'feature_fraction': feature_fraction,
}

model = lgb.LGBMClassifier(verbose=-1, verbosity=-1)
random_search = RandomizedSearchCV(
    model,
    param_dist,
    cv=5,
    n_iter=20,
    verbose=10,
    n_jobs=-1,
    scoring="neg_log_loss",
)
random_search.fit(train.iloc[:1000], target[:1000])

In [ ]:
random_search.best_estimator_, random_search.best_score_

# HalvingGridSearch

In [ ]:
%%time

from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

param_dist = {
    "max_depth": max_depth,
    "min_child_weight": [1, 3, 6],
    "n_estimators": n_estimators,
    "learning_rate": np.linspace(0.01, 0.3, 3),
   # 'device_type': ['gpu']
   'bagging_fraction': bagging_fraction,
   'bagging_freq': bagging_freq,
   'feature_fraction': feature_fraction,
}

model = lgb.LGBMClassifier(verbose=-1, verbosity=-1)
halving_grid_search = HalvingGridSearchCV(
    model, param_dist, cv=5, scoring="neg_log_loss", verbose=1, n_jobs=-1, min_resources=50
)
halving_grid_search.fit(train.iloc[:1000], target[:1000])

In [ ]:
halving_grid_search.best_estimator_, halving_grid_search.best_score_

# BayesianOptimization

In [ ]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 4.1 MB/s 


In [ ]:
# can use skopt https://scikit-optimize.github.io/stable/auto_examples/bayesian-optimization.html

In [ ]:
#from skopt import BayesSearchCV

In [ ]:
# Bayesian Optimization is popular, it will pick the range around the best parameter range

In [ ]:
#https://colab.research.google.com/github/yandexdataschool/mlhep2019/blob/master/notebooks/day-8/NN_optimisation/3-bayesian_optimization.ipynb#scrollTo=6mH_S_RfsRo-

In [ ]:
#https://towardsdatascience.com/bayesian-optimization-for-hyperparameter-tuning-how-and-why-655b0ee0b399

In [ ]:
import time
import warnings

warnings.filterwarnings("ignore")
#from bayes_opt import BayesianOptimization
#import BayesianOptimization
#from bayes_opt import BayesianOptimization

def lgb_eval(max_depth, min_child_weight, learning_rate):
    params = {"metric": "auc", "verbose": -1}

    params["max_depth"] = int(max(max_depth, 1))
    params["min_child_weight"] = int(max(min_child_weight, 1))
    params["learning_rate"] = float(min(learning_rate, 1))
    params['objective'] = "binary"
    cv_result = lgb.cv(
        params,
        lgb.Dataset(train.iloc[:1000], label=target[:1000]),
        nfold=3,
        num_boost_round=2000,
        seed=0,
        stratified=True,
        )
    return 1.0 * np.array(cv_result["auc-mean"]).max()


lgbBO = BayesSearchCV(
    f=lgb_eval,
    pbounds={
        "max_depth": (3, 9),
        "min_child_weight": (1, 20),
        "learning_rate": (0.05, 0.5),
    })

lgbBO.maximize(init_points=10, n_iter=20, acq="ei", alpha=1e-3)
print(lgbBO.max)